### Activity: Create interactive temporal visualization  

In this activity we will combine all the knowledge of time-series manipulation and range-slider learned in
previous exercises. 
Interactive slider is commnly used in time-series visualization

**import the required libraries**

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from pathlib import Path
import pandas as pd
import numpy as np
from ipywidgets import interact
%matplotlib inline

**setup the output to jupyter notebook**

In [ ]:
DATA_PATH = Path('../datasets/chap5_data/')
output_notebook()

**create a dataframe `microsoft_df` and parse the date column**

In [ ]:
microsoft_df = pd.read_csv(DATA_PATH / "microsoft_stock.csv", parse_dates=['date'])
microsoft_df.head()

**Set the index as date**

In [ ]:
microsoft_df.index = microsoft_df.date

**create `date` numpy array and `source` as ColumnDataSource. We will use these to draw line plot.**

In [ ]:
dates = np.array(microsoft_df['date'], dtype=np.datetime64)
source = ColumnDataSource(data=dict(date=dates, close=microsoft_df['high']))

**initialize the figure and drwa the line**

In [ ]:
p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None, title="Time Series Stock Data",
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#ffefef", x_range=(dates[1000], dates[1800]))

r = p.line('date', 'close', source=source)
p.yaxis.axis_label = 'High Price'


**Create range slider using `RangeTool`**

In [ ]:
select = figure(title="Drag the Slider",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#ffefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "green"
range_tool.overlay.fill_alpha = 0.2


**Write a custom update function which aggregate data by `month`, `year` and `day`**

In [ ]:
def update(f):
    if   f == "day":
        r.data_source.data = dict({
            'date': microsoft_df.index,
            'close': microsoft_df.high
        })
    elif f == "month":
        month = microsoft_df.groupby(pd.Grouper(freq="M"))[['high']].mean()
        r.data_source.data = dict({
            'date': month.index,
            'close': month.high
        })
    elif f == "year":
        year = microsoft_df.groupby(pd.Grouper(freq="Y"))[['high']].mean()
        r.data_source.data = dict({
            'date': year.index,
            'close': year.high
        })
    
    push_notebook()

**Plot both range slider and aggregator on figure**

In [ ]:
select.line('date', 'close', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select), notebook_handle=True)
interact(update, f=["day", "month", "year"])